In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import cv2
import os

In [34]:
count = 0

cap = cv2.VideoCapture('/content/drive/MyDrive/Demo_Video/Real_face/zcijgqopyg.mp4')
success, image = cap.read()

In [35]:
success

True

In [36]:
if not os.path.exists('/content/drive/MyDrive/Demo_Video/frames'):
    os.mkdir('/content/drive/MyDrive/Demo_Video/frames')

In [37]:
while success:
    success, image = cap.read()

    if not success:
        break
    
    cv2.imwrite('/content/drive/MyDrive/Demo_Video/frames/' + str(count) + '.jpg' , image)
    count += 1

In [38]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [39]:
pred_path='/content/drive/MyDrive/Demo_Video/frames'
classes = ['FAKE!!', 'REAL!!']

In [40]:
class network(nn.Module):
    def __init__(self):
        super(network,self).__init__()
        self.keep = 0.5

        self.conv1=nn.Conv2d(in_channels=3,out_channels=8,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=8)
        self.relu1=nn.ReLU()
        self.pool1=nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1,padding=1)
        self.bn2=nn.BatchNorm2d(num_features=16)
        self.relu2=nn.ReLU()

        self.conv3=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()

        self.pool2=nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.conv4=nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3,stride=1,padding=1)
        self.bn4=nn.BatchNorm2d(num_features=16)
        self.relu4=nn.ReLU()
        

        self.flatten = nn.Flatten()

        self.fc=nn.Linear(in_features=33*33*16,out_features=625, bias=True)
        self.relu5=nn.ReLU()
        self.dout=nn.Dropout(p=1 - self.keep)

        self.outputs = torch.nn.Linear(625, 2, bias=True)
        
                
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool1(output)
            
        output=self.conv2(output)
        output=self.bn2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)

        output=self.pool2(output)

        output=self.conv4(output)
        output=self.bn4(output)
        output=self.relu4(output)

        output=self.flatten(output)
                        
        #output=output.view(-1,32*75*75)
        output=self.fc(output)


        output=self.relu5(output)
        output=self.dout(output)
            
            
        #output=self.fc(output)
        output=self.outputs(output) 

        return output

In [41]:
model = network()
model.load_state_dict(torch.load('/content/drive/MyDrive/Models/mxepmodel.pth', map_location="cpu"))
model.eval()

network(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=17424, out_fea

In [42]:
transformer=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], 
                        [0.5,0.5,0.5])
])

In [43]:
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [44]:
images_path=glob.glob(pred_path+'/*.jpg')

In [45]:
pred_dict={}

for i in images_path:
    pred_dict[i[41:]]=prediction(i,transformer)

In [46]:
pred_dict

{'0.jpg': 'REAL!!',
 '1.jpg': 'REAL!!',
 '10.jpg': 'FAKE!!',
 '100.jpg': 'REAL!!',
 '101.jpg': 'REAL!!',
 '102.jpg': 'REAL!!',
 '103.jpg': 'REAL!!',
 '104.jpg': 'REAL!!',
 '105.jpg': 'REAL!!',
 '106.jpg': 'FAKE!!',
 '107.jpg': 'REAL!!',
 '108.jpg': 'REAL!!',
 '109.jpg': 'REAL!!',
 '11.jpg': 'FAKE!!',
 '110.jpg': 'REAL!!',
 '111.jpg': 'REAL!!',
 '112.jpg': 'REAL!!',
 '113.jpg': 'REAL!!',
 '114.jpg': 'REAL!!',
 '115.jpg': 'REAL!!',
 '116.jpg': 'REAL!!',
 '117.jpg': 'REAL!!',
 '118.jpg': 'REAL!!',
 '119.jpg': 'REAL!!',
 '12.jpg': 'REAL!!',
 '120.jpg': 'REAL!!',
 '121.jpg': 'REAL!!',
 '122.jpg': 'REAL!!',
 '123.jpg': 'REAL!!',
 '124.jpg': 'REAL!!',
 '125.jpg': 'REAL!!',
 '126.jpg': 'REAL!!',
 '127.jpg': 'REAL!!',
 '128.jpg': 'REAL!!',
 '129.jpg': 'REAL!!',
 '13.jpg': 'REAL!!',
 '130.jpg': 'REAL!!',
 '131.jpg': 'REAL!!',
 '132.jpg': 'REAL!!',
 '133.jpg': 'REAL!!',
 '134.jpg': 'REAL!!',
 '135.jpg': 'REAL!!',
 '136.jpg': 'FAKE!!',
 '137.jpg': 'FAKE!!',
 '138.jpg': 'REAL!!',
 '139.jpg': 'FAKE!

In [47]:
r=0
f=0
for k, i in pred_dict.items():
  if i=='FAKE!!':
    f+=1
  else: r+=1 

In [48]:
r

136

In [49]:
f

11